# Except for some extremely minor revisions, all of the credit for this notebook goes to Matteo Esposito!

In [1]:
import numpy as np
import camb

In [2]:
model0 = {
    'ombh2' : 0.022445,
    'omch2' : 0.120567,
    'n_s' : 0.96,
    'A_s' : 2.12723788013000E-09,
    'h' : 0.67,
    'w0' : -1.00,
    'wa' : 0.00,
    'OmK' : 0.00
}          

model0_zs = [
    2.000000,
    1.000000,
    0.570000,
    0.300000,
    0.000000
]

In [9]:
spectra.cosm.iloc[0]

NameError: name 'spectra' is not defined

In [3]:
def get_camb_Pk(mlc, omnuh2_in, nu_massive=False, zs=[0], nnu_massive_in=1):
    pars = camb.CAMBparams()
    omch2_in = mlc["omch2"]

    print(mlc)
    
    mnu_in = 0
    nnu_massive = 0
    h = mlc["h"]
   
    if nu_massive:
        mnu_in = omnuh2_in * camb.constants.neutrino_mass_fac / \
            (camb.constants.default_nnu / 3.0) ** 0.75 
        #print("The mnu value", mnu_in, "corresponds to the omnuh2 value",
        #    omnuh2_in)
        omch2_in -= omnuh2_in
        nnu_massive = nnu_massive_in

    pars.set_cosmology(
        H0=h * 100,
        ombh2=mlc['ombh2'],
        omch2=omch2_in,
        mnu=mnu_in,
        num_massive_neutrinos=nnu_massive,
        omk=mlc['OmK'],
        tau=0.0952,
        neutrino_hierarchy="degenerate"
    )
    
    pars.InitPower.set_params(As=mlc['A_s'], ns=mlc['n_s'],
        r=0, nt=0.0, ntrun=0.0)
    pars.NonLinear = camb.model.NonLinear_none
    pars.WantCls = False
    pars.WantScalars = False
    pars.Want_CMB = False
    pars.DoLensing = False
    pars.YHe = 0.24
    pars.set_accuracy(AccuracyBoost=2)
    pars.Accuracy.AccuracyBoost = 3
    pars.Accuracy.lAccuracyBoost = 3
    pars.Accuracy.AccuratePolarization = False
    pars.Transfer.kmax = 20.0 / h
    
    if mlc['w0'] != -1 or mlc['wa'] != 0:
        pars.set_dark_energy(w=w0, wa=wa, dark_energy_model='ppf') 
        
    # Lukas. I added the "/ h"
    # Lukas. I've verified that for model 0, these redshifts are correct.
    pars.set_matter_power(redshifts=zs, kmax=20.0 / h, nonlinear=False)    
        
    results = camb.get_results(pars)

    sigma12 = results.get_sigmaR(12, hubble_units=False)
    
    # Lukas: I changed var1 and var2 to 8, they were both 2 before
    # Lukas: I changed npoints to 100000, it was 200 before
    kh, z, pkh = results.get_matter_power_spectrum(
        minkh=1e-4/h, maxkh=10.0 / h, npoints = 100000,
        var1=8, var2=8
    ) 
    
    return {'kk': kh*h, 'zz': z, 'Pk': pkh, 's12': sigma12}

In [4]:
omnu_Lukas = 0.002148659574468

LCDM = get_camb_Pk(model0, omnuh2_in=omnu_Lukas, nu_massive=False,
    zs=model0_zs)

mnu_Lukas = 0.1998324308090579 # this is intended to mimic a physical neutrino
# density of 0.002148659574468
mnu_Matteo = 0.3
nuCDM = get_camb_Pk(model0, omnuh2_in=omnu_Lukas, nu_massive=True,
    zs=model0_zs)

{'ombh2': 0.022445, 'omch2': 0.120567, 'n_s': 0.96, 'A_s': 2.12723788013e-09, 'h': 0.67, 'w0': -1.0, 'wa': 0.0, 'OmK': 0.0}
{'ombh2': 0.022445, 'omch2': 0.120567, 'n_s': 0.96, 'A_s': 2.12723788013e-09, 'h': 0.67, 'w0': -1.0, 'wa': 0.0, 'OmK': 0.0}


In [5]:
np.save('LCDM_tiny.npy', LCDM, allow_pickle=True)
np.save('nuCDM_tiny.npy', nuCDM, allow_pickle=True)

In [10]:
def kzps(mlc, omnuh2_in, nu_massive=False, zs = [0], nnu_massive_in=1):
    """
    Returns the scale axis, redshifts, power spectrum, and sigma12
    of a Lambda-CDM model
    @param mlc : "MassLess Cosmology"
        a dictionary of values
        for CAMBparams fields
    @param omnuh2_in : neutrino physical mass density
    @nu_massive : if this is True,
        the value in omnuh2_in is used to set omnuh2.
        If this is False,
        the value in omnuh2_in is simply added to omch2.

    Possible mistakes:
    A. We're setting "omk" with OmK * h ** 2. Should I have used OmK? If so,
        the capitalization here is nonstandard.
    """ 
    print("redshifts", zs)
    print("baryon dens", mlc['ombh2'])
    print("cdm dens", mlc['omch2'])
    print("spec index", mlc["n_s"])
    print("scalar amp", mlc["A_s"])
    print("Hubble con", mlc["h"])
    print("DE con", mlc["w0"])
    print("DE slope", mlc['wa'])
    print("curvie", mlc['OmK'])
    
    pars = camb.CAMBparams()
    omch2_in = mlc["omch2"]

    mnu_in = 0
    nnu_massive = 0
    h = mlc["h"]

    if nu_massive:
        '''This is a horrible workaround, and I would like to get rid of it
        ASAP. It destroys dependence on TCMB and
        neutrino_hierarchy, possibly more. But CAMB does not accept omnuh2 as
        an input, so I have to reverse-engineer it somehow.
        
        Also, should we replace default_nnu with something else in the
        following expression? Even if we're changing N_massive to 1,
        N_total_eff = 3.046 nonetheless, right?'''
        mnu_in = omnuh2_in * camb.constants.neutrino_mass_fac / \
            (camb.constants.default_nnu / 3.0) ** 0.75 
        #print("The mnu value", mnu_in, "corresponds to the omnuh2 value",
        #    omnuh2_in)
        omch2_in -= omnuh2_in
        nnu_massive = nnu_massive_in

    # tau is a desperation argument
    pars.set_cosmology(
        H0=h * 100,
        ombh2=mlc["ombh2"],
        omch2=omch2_in,
        omk=mlc["OmK"],
        mnu=mnu_in,
        #num_massive_neutrinos=nnu_massive, CODE_BLUE
        tau=0.0952, # just like in Matteo's notebook, at least (but maybe I got
            # this value from somewhere else...)
        neutrino_hierarchy="degenerate" # 1 eigenstate approximation; our
        # neutrino setup (see below) is not valid for inverted/normal
        # hierarchies.
    )
    '''
    # Matteo really didn't use any of this block? I don't understand--this
    # block seems well theoretically motivated.
    pars.num_nu_massless = 3.046 - nnu_massive
    pars.nu_mass_eigenstates = nnu_massive
    stop_i = pars.nu_mass_eigenstates + 1
    pars.nu_mass_numbers[:stop_i] = \
        list(np.ones(len(pars.nu_mass_numbers[:stop_i]), int))
    pars.num_nu_massive = 0
    if nnu_massive != 0:
        pars.num_nu_massive = sum(pars.nu_mass_numbers[:stop_i])
    '''
    pars.InitPower.set_params(As=mlc["A_s"], ns=mlc["n_s"],
        r=0, nt=0.0, ntrun=0.0) # the last three are desperation arguments
    
    ''' The following seven lines are desperation settings
    If we ever have extra time, we can more closely study what each line does
    '''
    # This is a desperation line in light of the previous line. The previous
    # line seems to have served me well enough so far, but BSTS.
    pars.NonLinear = camb.model.NonLinear_none
    pars.WantCls = False
    pars.WantScalars = False
    pars.Want_CMB = False
    pars.DoLensing = False
    pars.YHe = 0.24
    # Matteo used this line but Andrea uses the following lines
    pars.set_accuracy(AccuracyBoost=2)
    # I already verified that commenting-out the next four lines DOES NOT
    # impact the Lukas-Matteo Gap.
    pars.Accuracy.AccuracyBoost = 3
    pars.Accuracy.lAccuracyBoost = 3
    pars.Accuracy.AccuratePolarization = False
    pars.Transfer.kmax = 20.0 / h

    # desperation if statement
    # CODE_GREEN should we add the additional try/catch that Matteo uses?
    if mlc["w0"] != -1 or float(mlc["wa"]) != 0:
        pars.set_dark_energy(w=mlc["w0"], wa=float(mlc["wa"]),
            dark_energy_model='ppf')
    
    ''' To change the the extent of the k-axis, change the following line as
    well as the "get_matter_power_spectrum" call. '''
    pars.set_matter_power(redshifts=zs, kmax=20.0 / h, nonlinear=False)
    
    results = camb.get_results(pars)

    # WHY DOESN'T THIS MAKE ANY DIFFERENCE???
    #results.calc_power_spectra(pars)

    sigma12 = results.get_sigmaR(12, hubble_units=False)
    
    '''
    In some cursory tests, the accurate_massive_neutrino_transfers
    flag did not appear to significantly alter the outcome.
    
    The flags var1=8 and var2=8 indicate that we are looking at the
    power spectrum of CDM + baryons (i.e. neutrinos excluded).
    '''
    k, z, p = results.get_matter_power_spectrum(
        minkh=1e-4 / h, maxkh=10.0 / h, npoints = 100000,
        var1=8, var2=8
    )
   
    # De-nest for the single-redshift case:
    if len(p) == 1:
        p = p[0] 
    return k, z, p, sigma12 

In [11]:
k0, z0, p0, s0 = kzps(model0, omnuh2_in=omnu_Lukas, nu_massive=False,
    zs=model0_zs)
knu, znu, pnu, snu = kzps(model0, omnuh2_in=omnu_Lukas, nu_massive=True,
    zs=model0_zs)


mimsims = {
    'no': {
        'k': k0 * model0['h'],
        'p': p0 / model0['h'] ** 3
    },
    'nu': {
        'k': knu * model0['h'],
        'p': pnu / model0['h'] ** 3
    }
}

redshifts [2.0, 1.0, 0.57, 0.3, 0.0]
baryon dens 0.022445
cdm dens 0.120567
spec index 0.96
scalar amp 2.12723788013e-09
Hubble con 0.67
DE con -1.0
DE slope 0.0
curvie 0.0
redshifts [2.0, 1.0, 0.57, 0.3, 0.0]
baryon dens 0.022445
cdm dens 0.120567
spec index 0.96
scalar amp 2.12723788013e-09
Hubble con 0.67
DE con -1.0
DE slope 0.0
curvie 0.0


In [12]:
np.save('mimsims.npy', mimsims, allow_pickle=True)